In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import xml.etree.ElementTree as ET
import tensorflow as tf
%matplotlib inline

# Object Detection API with TensorFlow 2

#### API setup

Download tensorflow 2 object detection api
https://github.com/tensorflow/models/tree/master/research/object_detection

In [ ]:
# tensorflow-gpu 2.10.1
# https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2.md

In [ ]:
!git clone https://github.com/tensorflow/models.git

In [ ]:
%cd C:\Users\Imtiaz\Downloads\DL_models\models\research

In [ ]:
# Compile protos.
!C:\Users\Imtiaz\Downloads\DL_models\protobuf\bin\protoc.exe object_detection/protos/*.proto --python_out=.

In [ ]:
# Install TensorFlow Object Detection API.
'''
copy setup.py from object_detection/packages/tf2 inside models/research
'''
!copy object_detection/packages/tf2/setup.py .

In [ ]:
!python -m pip install .

In [ ]:
# Test the installation.
!python object_detection/builders/model_builder_tf2_test.py

# Transforming xml files into one csv file

In [ ]:
TRAIN_ANNOT_PATH  = ""
VALID_ANNOT_PATH  = ""
TEST_ANNOT_PATH  = ""

In [ ]:
from utils.xml_to_csv import generate_csv_file
TRAIN_CSV = "dataset_csv/annotations.csv"
VALID_CSV = "dataset_csv/annotations_valid.csv"
TEST_CSV  = "dataset_csv/test_data.csv"
generate_csv_file(TRAIN_ANNOT_PATH, TRAIN_CSV)
generate_csv_file(VALID_ANNOT_PATH, VALID_CSV)
generate_csv_file(TEST_ANNOT_PATH, TEST_CSV)

In [ ]:
TRAIN_CSV = "dataset_csv/annotations.csv"
VALID_CSV = "dataset_csv/annotations_valid.csv"
TEST_CSV  = "dataset_csv/test_data.csv"
train_df = pd.read_csv(TRAIN_CSV)
valid_df = pd.read_csv(VALID_CSV)
test_df  = pd.read_csv(TEST_CSV)

In [ ]:
train_df.head()

In [ ]:
valid_df.head()

In [ ]:
test_df.head()

In [ ]:
def show_csv(df, indx: list):
    df = df.reset_index(drop=True)
    length = len(indx)
    cnt = 0
    if length > 2:
        r, c = length//3, 4
        _, axis = plt.subplots(r, c, figsize=(16, 10))
        for i in range(r):
            for j in range(c):
                if cnt > length-1:
                    break
                img = df.iloc[indx[cnt], :]
                file_name = img['filename']
                gImg = df[df['filename'] == file_name].reset_index(drop=True)
                image = cv2.cvtColor(cv2.imread(file_name), cv2.COLOR_BGR2RGB)
                for z in range(gImg.shape[0]):
                    class_, xmin, ymin, xmax, ymax = gImg.iloc[z, 3:]
                    cv2.rectangle(image, (xmin, ymin),
                                  (xmax, ymax), (0, 255, 0), 2)
                    cv2.putText(image, f"{class_}", (xmin, ymin + 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                if r == 1:
                    axis[j].imshow(image)
                    axis[j].set_xticks([]), axis[j].set_yticks([])
                else:
                    axis[i, j].imshow(image)
                    axis[i, j].set_xticks([]), axis[i, j].set_yticks([])
                cnt += 1

    else:
        r, c = 1, 2
        _, axis = plt.subplots(r, c, figsize=(12, 6))
        for i in range(length):
            img = df.iloc[indx[i], :]
            file_name = img['filename']
            gImg = df[df['filename'] == file_name].reset_index(drop=True)
            image = cv2.cvtColor(cv2.imread(file_name), cv2.COLOR_BGR2RGB)
            for z in range(gImg.shape[0]):
                class_, xmin, ymin, xmax, ymax = gImg.iloc[z, 3:]
                cv2.rectangle(image, (xmin, ymin),
                              (xmax, ymax), (0, 255, 0), 2)
                cv2.putText(image, f"{class_}", (xmin, ymin + 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            axis[i].imshow(image)
            axis[i].set_xticks([]), axis[i].set_yticks([])
            
show_csv(train_df, [1010,900,250,600])


In [ ]:
show_csv(test_df, [0,1,2,3])

In [ ]:
show_csv(valid_df, [0, 3, 7, 2])

# Generate tfrecords

In [ ]:
%cd C:\Users\Imtiaz\Downloads\DL_models\utils

In [ ]:
ls

In [ ]:
# train set
!python generate_tfrecords.py --path_to_images C:\Users\Imtiaz\Downloads\DL_models\data\images\train --path_to_annot C:\Users\Imtiaz\Downloads\DL_models\dataset_csv\annotations.csv --path_to_label_map C:\Users\Imtiaz\Downloads\DL_models\annotations\labelmap.pbtxt --path_to_save_tfrecords C:\Users\Imtiaz\Downloads\DL_models\annotations\train.record

In [ ]:
# validation set
!python generate_tfrecords.py --path_to_images C:\Users\Imtiaz\Downloads\DL_models\data\images\val --path_to_annot C:\Users\Imtiaz\Downloads\DL_models\dataset_csv\annotations_valid.csv --path_to_label_map C:\Users\Imtiaz\Downloads\DL_models\annotations\labelmap.pbtxt --path_to_save_tfrecords C:\Users\Imtiaz\Downloads\DL_models\annotations\val.record

In [ ]:
# test set
!python generate_tfrecords.py --path_to_images C:\Users\Imtiaz\Downloads\DL_models\data\images\test --path_to_annot C:\Users\Imtiaz\Downloads\DL_models\dataset_csv\test_data.csv --path_to_label_map C:\Users\Imtiaz\Downloads\DL_models\annotations\labelmap.pbtxt --path_to_save_tfrecords C:\Users\Imtiaz\Downloads\DL_models\annotations\test.record

In [ ]:
train_tfrecord_file = 'C:\\Users\\Imtiaz\\Downloads\\DL_models\\annotations\\train.record'
val_tfrecord_file = 'C:\\Users\\Imtiaz\\Downloads\\DL_models\\annotations\\val.record'
test_tfrecord_file = 'C:\\Users\\Imtiaz\\Downloads\\DL_models\\annotations\\test.record'
label_map = 'C:\\Users\\Imtiaz\\Downloads\\DL_models\\annotations\\labelmap.pbtxt'
# Create a TFRecordDataset
train_dataset = tf.data.TFRecordDataset([train_tfrecord_file])
val_dataset = tf.data.TFRecordDataset([val_tfrecord_file])
test_dataset = tf.data.TFRecordDataset([test_tfrecord_file])

In [ ]:
for record in val_dataset.take(2):
    example = tf.train.Example()
    example.ParseFromString(record.numpy())
    print(example)

In [ ]:
import keras_cv
from keras_cv import bounding_box
from keras_cv import visualization
def parse_tfrecord_fn(example):
    feature_description = {
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'image/height': tf.io.FixedLenFeature([], tf.int64),
        'image/width': tf.io.FixedLenFeature([], tf.int64),
        'image/object/bbox/xmin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/xmax': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymax': tf.io.VarLenFeature(tf.float32),
        'image/object/class/label': tf.io.VarLenFeature(tf.int64),
    }
    
    parsed_example = tf.io.parse_single_example(example, feature_description)

    # Decode the JPEG image and normalize the pixel values to the [0, 1] range.
    img = tf.image.decode_jpeg(parsed_example['image/encoded'], channels=3) # Returned as uint8
    # Normalize the pixel values to [0, 256]
    img = tf.image.convert_image_dtype(img, tf.uint8)

    # Get the bounding box coordinates and class labels.
    xmin = tf.sparse.to_dense(parsed_example['image/object/bbox/xmin'])
    xmax = tf.sparse.to_dense(parsed_example['image/object/bbox/xmax'])
    ymin = tf.sparse.to_dense(parsed_example['image/object/bbox/ymin'])
    ymax = tf.sparse.to_dense(parsed_example['image/object/bbox/ymax'])
    labels = tf.sparse.to_dense(parsed_example['image/object/class/label'])

    # Stack the bounding box coordinates to create a [num_boxes, 4] tensor.
    rel_boxes = tf.stack([xmin, ymin, xmax, ymax], axis=-1)
    boxes = keras_cv.bounding_box.convert_format(rel_boxes, source='rel_xyxy', target='xyxy', images=img)

    # Create the final dictionary.
    image_dataset = {
        'images': img,
        'bounding_boxes': {
            'classes': labels,
            'boxes': boxes
        }
    }

    return image_dataset

# train_dataset = train_dataset.map(parse_tfrecord_fn)
